# **Pokémon Diffusion<a id="top"></a>**

> #### ``04-Training-Diffusion-Model.ipynb``

<i><small>**Alumno:** Alejandro Pequeño Lizcano<br>Última actualización: 11/03/2024</small></i></div>

### Sampling

Una vez que hemos establecido todos los pasos para el preprocesamiento de datos y definido la arquitectura del modelo de difusión, avanzamos hacia la etapa de entrenamiento del modelo. En esta fase, hemos desarrollado funciones auxiliares destinadas a visualizar los resultados del modelo a medida que se lleva a cabo el entrenamiento y también para poder visualizar los resultados finales del modelo una vez que se ha completado el entrenamiento.

Iniciamos con la función ``sampling()``, la cual despliega muestras conforme el modelo se entrena y nos proporciona una herramienta fundamental para evaluar y visualizar el rendimiento del modelo a medida que evoluciona a lo largo del tiempo y la difusión inversa. En primer lugar, se establece $\beta$ y los valores correspondientes de $\alpha$ que desempeñan un papel esencial en la difusión inversa. Posteriormente, se inicializa el ruido, dando inicio al proceso a lo largo de los pasos de difusión. Cada iteración implica la normalización del tiempo, la generación de ruido y la predicción del modelo. La imagen final es obtenida después de aplicar la resta del ruido predicho en un instante de tiempo $t$ a la imagen en ese mismo instante, dando como resultado la imagen en el instante $t-1$. Este proceso se repite hasta que se obtiene la imagen original: $x_{0}$.

Dicha función sigue el algoritmo 2 de [Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2006.11239) y se ha modificado para que sea capaz de generar imágenes condicionadas a una etiqueta. Para ello, se ha añadido el parámetro ``label`` al modelo ``build_ddpm_model()``. Esto permite que el modelo de difusión genere imágenes de un tipo de pokemon concreto.

<div style="text-align:center">
<img src="https://miro.medium.com/v2/resize:fit:1048/1*UxxYjCEfV99i-drcTZp87w.png" width="40%" height="30%" />
</div>

In [ ]:
# Algorithm 2: Sampling
# =====================================================================
def sampling(
    model: tf.keras.models.Model,
    start_noise: np.ndarray,
    T: int = T,
    scheduler: str = "linear",
    beta_start: float = beta_start,
    beta_end: float = beta_end,
) -> np.ndarray:
    """
    Samples an image from the model.

    :param model: The model to sample from.
    :param start_noise: The noise to start the sampling from.
    :param T: The number of timesteps to sample for.
    :param scheduler: The type of schedule to use. Options are "linear" or "cosine".
    :param beta_start: Starting value of beta.
    :param beta_end: Ending value of beta.
    :return: The sampled image.
    """

    # Get the beta schedule and corresponding alpha values
    beta = beta_scheduler(scheduler, T, beta_start, beta_end)
    alpha = 1.0 - beta
    alpha_cumprod = np.cumprod(alpha)

    # Set the starting noise
    x_t = start_noise  # 1: x_T ~ N(0, I)

    # Reverse the diffusion process
    for t in tqdm(
        reversed(range(1, T)), desc="Sampling", total=T - 1, leave=False
    ):  # 2: for t = T − 1, . . . , 1 do
        # Compute normalized timestep
        normalized_t = np.array([t / T]).reshape(1, -1).astype("float32")
        # Sample z_t
        z = (
            np.random.normal(size=x_t.shape)
            if t > 1
            else np.zeros(x_t.shape).astype("float32")
        )  # 3: z ∼ N(0, I) if t > 1, else z = 0
        # Calculate x_(t-1)
        predicted_noise = model.predict(
            [x_t, normalized_t], verbose=0
        )  # Predict the noise estimate using the model = eps_theta
        x_t = (
            x_t - (1 - alpha[t]) / np.sqrt(1 - alpha_cumprod[t]) * predicted_noise
        ) / np.sqrt(alpha[t]) + np.sqrt(
            beta[t]
        ) * z  # 4: x_(t-1) = (x_t - (1 - alpha_t) / sqrt(1 - alpha_cumprod_t) * eps_theta) / sqrt(alpha_t) + sigma_t * z

    # Return the final sample
    return x_t  # 5: return x_0

Las funciones auxiliares proporcionadas cumplen roles importantes para la evaluación y visualización del modelo de difusión:

- ``generate_em()``: Esta función crea una etiqueta aleatoria para condicionar el modelo de difusión. Genera un vector de ceros de longitud num_classes y asigna el valor 1 en una posición aleatoria dentro del vector.

- ``plot_samples()``: visualiza muestras generadas por el modelo de difusión a partir de las funciones mencionadas anteriormente.

In [ ]:
# Auxiliary functions
# =====================================================================


# Generate a random embedding (label) =====================================================================
def generate_em(num_classes: int = NUM_CLASSES) -> np.ndarray:
    """
    Generates a random embedding (label)
    :param num_classes: The number of classes
    """
    em = np.zeros(num_classes)
    em[np.random.randint(0, num_classes - 1)] = 1
    return em


# Plot samples function =====================================================================
def plot_samples(
    model: tf.keras.models.Model,
    num_samples: int = 2,
    T: int = T,
    scheduler: str = "linear",
    beta_start: float = beta_start,
    beta_end: float = beta_end,
) -> None:
    """
    Plots samples from the model.

    :param model: The model to sample from.
    :param num_samples: The number of samples to plot.
    :param T: The number of timesteps to sample for.
    :param scheduler: The type of schedule to use. Options are "linear" or "cosine".
    :return: The sampled image.
    """

    fig, axs = plt.subplots(
        1, num_samples, figsize=(num_samples * 2, 2)
    )  # Creating a row of subplots

    for i in trange(num_samples, desc="Sample plot", leave=True):
        start_noise = np.random.normal(size=(1, IMG_SIZE, IMG_SIZE, 3)).astype(
            "float32"
        )
        y_label = generate_em().reshape(
            1, 18
        )  # reshape to (1,18) to match the model input
        sample = sampling(
            model, start_noise, y_label, T, scheduler, beta_start, beta_end
        )
        sample = (sample + 1.0) / 2.0  # Scale to [0, 1]
        axs[i].imshow(sample[0])
        axs[i].title.set_text(
            onehot_to_string(y_label[0])
        )  # use the onehot_to_string function described above
        axs[i].axis("off")

    plt.show()

### Training

Como paso final, se procede a entrenar el modelo de difusión. Para ello, se ha definido la función ``training()`` que engloba todo el proceso de difusión completo, tanto hacia adelante como hacia atrás y los ploteos de las muestras generadas. Para implementar el training hemos usado el **Algoritmo 1** de [Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2006.11239) y se ha modificado para que sea capaz de generar imágenes condicionadas a una etiqueta.

<div style="text-align:center">
<img src="https://miro.medium.com/v2/resize:fit:1400/1*XbNgIrfo269LT9QKGcg2lw.png" width="40%" height="30%" />
</div>

Tambiñen se han añadido unas funcionalidades extra que permiten guardar cada epoch el modelo y sus pesos en un fichero con extensión .h5. Esto se hace para poder cargar el modelo y continuar el entrenamiento desde donde se quedó en caso de que se interrumpa por algún motivo.

<span style="color: red; font-size: 1.5em;">&#9888;</span> <i><small>**NOTA:** Por cada epoch se guarda en un fichero con extensión .h5 tanto el modelo como sus pesos. Este proceso se realiza ya que todo el entrenamiento es muy costoso y si se interrumpe por algún motivo, se puede volver a cargar el modelo y continuar el entrenamiento desde donde se quedó.

También cabe destacar que para una mayor eficiencia en el entrenamiento, se ha optado por realizar el ``sampling()`` cada 5 epochs.
</small></i>

In [ ]:
# Algorithm 1: Training
# =====================================================================
def training(
    model: tf.keras.models.Model,
    dataset: tf.data.Dataset,
    optimizer: tf.keras.optimizers.Optimizer,
    loss_fn: tf.keras.losses.Loss,
    total_epochs: int = 10,
    scheduler: str = "cosine",
    T: int = 100,
) -> None:
    """
    Performs the training loop.

    :param model: The model to train.
    :param dataset: The training dataset.
    :param optimizer: The optimizer to use.
    :param loss_fn: The loss function to use.
    :param total_epochs: The number of epochs to train for.
    :param scheduler: The type of schedule to use. Options are "linear" or "cosine".
    :param T: The number of timesteps to sample for.
    :return: None
    """

    # Save intermodels by epoch
    # =====================================================================

    # Create the folder to save the models by epoch
    folder_epoch = f"../../models/inter_models/diffusion_{IMG_SIZE}_{BATCH_SIZE}_{EPOCHS}_{T}_{scheduler}_ddpm"
    if not os.path.exists(folder_epoch):
        os.makedirs(folder_epoch)

    # Check if there are checkpoints to load
    if len(glob.glob(f"{folder_epoch}/diffusion_{scheduler}_*.h5")) > 0:

        last_checkpoint = sorted(
            glob.glob(f"{folder_epoch}/diffusion_{scheduler}*.h5"),
            key=lambda x: int(x.split("_")[-1].split(".")[0]),
        )[
            -1
        ]  # Get the last checkpoint
        print(f"Loading checkpoint {last_checkpoint}...")

        # Get the epoch from the checkpoint
        prev_epoch = int(
            last_checkpoint.split("_")[-1].split(".")[0]
        )  # Get the epoch from the checkpoint
        print(f"Resuming training from epoch {prev_epoch}...")

        model = tf.keras.models.load_model(
            f"{folder_epoch}/diffusion_{scheduler}_{prev_epoch}.h5"
        )  # Load the model

    else:
        prev_epoch = 0
        print("No checkpoints found, starting training from scratch...")

    # Start the training loop
    # =====================================================================

    # Get scheduler values
    beta = beta_scheduler(scheduler, T, beta_start, beta_end)  # Get beta
    alpha = 1.0 - beta  # Get alpha
    alpha_cumprod = np.cumprod(alpha)  # Get alpha cumulative product

    for epoch in trange(
        prev_epoch,
        total_epochs,
        desc=f"Training",
        total=total_epochs - prev_epoch,
        leave=True,
    ):  # 1: repeat (iterations through the epochs)
        for step, input_data in tqdm(
            enumerate(dataset),
            desc=f"Epoch {epoch+1}/{total_epochs}",
            total=len(dataset),
            leave=True,
        ):  # 1: repeat (iterations through the batches)
            # Generate a single timestep for one entire batch
            t = np.random.randint(0, T)
            normalized_t = np.full(
                (input_data.shape[0], 1), t / T, dtype=np.float32
            )  # 3: t ~ U(0, T)

            # Get the target noise
            noised_data = forward_diffusion(input_data, t, scheduler)  # 2: x_0 ~ q(x_0)
            target_noise = noised_data - input_data * np.sqrt(
                alpha_cumprod[t]
            )  # 4: eps_t ~ N(0, I)

            # 5: Take a gradient descent step on
            with tf.GradientTape() as tape:
                predicted_noise = model(
                    [noised_data, normalized_t], training=True
                )  # eps_theta -> model(x_t, t/T)
                loss = loss_fn(
                    target_noise, predicted_noise
                )  # gradient of the loss (MSE(eps_t, eps_theta))
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        print(f"EPOCH {epoch+1} LOSS: {loss.numpy():.4f} \n{'='*69}")

        # Save the model at the end of every 20 epochs
        if (epoch + 1) % 20 == 0:
            print(f"\tSaving model in epoch {epoch+1}")
            model.save(f"{folder_epoch}/diffusion_{scheduler}_{epoch+1}.h5")

        # Sample and plot every 10 epochs
        if (epoch + 1) % 10 == 0:
            print("\tSampling images...")
            plot_samples(model, num_samples=3, scheduler=scheduler, T=T)

In [ ]:
# Train the model
# =====================================================================
training(
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    loss_fn=loss_fn,
    scheduler="cosine",
    num_epochs=EPOCHS,
)

### Save Model

Finalmente, se guardan los resultados finales del modelo de difusión en un fichero `.h5` para su posterior uso y visualización. TODO: MIRAR OTROS FORMATOS DE GUARDADO

In [ ]:
# Save the model function
# =====================================================================
def save_model(model: tf.keras.models.Model, model_name: str) -> None:
    """Saves the model

    :param model: The model to save
    :param model_name: The name of the model
    :return: None
    """

    # Save the model
    model_dir = "./diffusion_models/models/"
    os.makedirs(model_dir, exist_ok=True)
    if not os.path.exists(os.path.join(model_dir, f"{model_name}.h5")):
        model.save(os.path.join(model_dir, f"{model_name}.h5"))
        print(f"Model {model_name}, saved successfully!")
    else:
        print(f"Model {model_name}, already exists!")

In [ ]:
# Save the model
model_name = f"diffusion_{IMG_SIZE}_{BATCH_SIZE}_{EPOCHS}_{T}_{scheduler}_ddpm"

save_model(model, model_name)